In [35]:
import requests
from bs4 import BeautifulSoup
import csv

# Send a GET request to the URL
url = "https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1"

# Create and open the first CSV file for the first set of fields
csv_file1 = open('movie_data1.csv', 'w', newline='', encoding='utf-8')
csv_writer1 = csv.writer(csv_file1)
csv_writer1.writerow(['Sno', 'Movie Name', 'Director1', 'Director2', 'Duration', 'Year', 'Ratings', 'Metascore'])

# Create and open the second CSV file for the second set of fields
csv_file2 = open('movie_data2.csv', 'w', newline='', encoding='utf-8')
csv_writer2 = csv.writer(csv_file2)
csv_writer2.writerow(['Movie Name', 'Star1', 'Star2', 'Star3', 'Star4', 'Votes', 'Genre1', 'Genre2', 'Genre3', 'Gross Collection', 'Popularity', 'Certification'])

page_num = 1
while True:
    # Send a GET request to the current page URL
    response = requests.get(url + "&start=" + str((page_num - 1) * 50))
    
    # Create a BeautifulSoup object to parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all movie items
    movies = soup.find_all('div', class_='lister-item-content')
    
    if not movies:
        # No more movies found, break the loop
        break
    
    # Iterate over each movie item and extract the desired information
    for movie in movies:
        # Extract fields for the first CSV file
        sno = movie.find('span', class_='lister-item-index unbold text-primary').text.strip('.')
        movie_name = movie.h3.a.text
        directors = movie.find_all('p')[2].find_all('a')
        director1 = directors[0].text if len(directors) > 0 else ''
        director2 = directors[1].text if len(directors) > 1 else ''
        duration = movie.find('span', class_='runtime').text.strip(' min')
        year = movie.find('span', class_='lister-item-year').text.strip('()')
        ratings = movie.find('div', class_='inline-block ratings-imdb-rating').strong.text
        metascore = movie.find('span', class_='metascore').text.strip() if movie.find('span', class_='metascore') else ''
        
        csv_writer1.writerow([sno, movie_name, director1, director2, duration, year, ratings, metascore])
        
        # Extract fields for the second CSV file
        stars = movie.find_all('p')[2].find_all('a')[2:]
        star1 = stars[0].text if len(stars) > 0 else ''
        star2 = stars[1].text if len(stars) > 1 else ''
        star3 = stars[2].text if len(stars) > 2 else ''
        star4 = stars[3].text if len(stars) > 3 else ''
        votes = movie.find('span', attrs={'name': 'nv'})['data-value']
        genres = movie.find('span', class_='genre').text.strip().split(', ')
        genre1 = genres[0] if len(genres) > 0 else ''
        genre2 = genres[1] if len(genres) > 1 else ''
        genre3 = genres[2] if len(genres) > 2 else ''
        gross = movie.find('span', class_='text-muted').find_all('span')[-1].text if movie.find('span', class_='text-muted').find_all('span') else ''
        popularity_element = movie.find('span', class_='sort-num_votes-visible')
        popularity = popularity_element.text.strip().split(' ')[-1] if popularity_element else ''
        certification = movie.find('span', class_='certificate').text if movie.find('span', class_='certificate') else ''
        
        csv_writer2.writerow([movie_name, star1, star2, star3, star4, votes, genre1, genre2, genre3, gross, popularity, certification])
    
    # Increment the page number for the next iteration
    page_num += 1

# Close the CSV files
csv_file1.close()
csv_file2.close()
